In [35]:
import logging
import requests
import pandas

log = logging.getLogger(__name__)

In [33]:
API_ROOT = "https://api.frankfurter.app/"

CURRENCIES = {
	"AUD": "Australian Dollar",
	"BGN": "Bulgarian Lev",
	"BRL": "Brazilian Real",
	"CAD": "Canadian Dollar",
	"CHF": "Swiss Franc",
	"CNY": "Chinese Renminbi Yuan",
	"CZK": "Czech Koruna",
	"DKK": "Danish Krone",
	"EUR": "Euro",
	"GBP": "British Pound",
	"HKD": "Hong Kong Dollar",
	"HUF": "Hungarian Forint",
	"IDR": "Indonesian Rupiah",
	"ILS": "Israeli New Sheqel",
	"INR": "Indian Rupee",
	"ISK": "Icelandic Króna",
	"JPY": "Japanese Yen",
	"KRW": "South Korean Won",
	"MXN": "Mexican Peso",
	"MYR": "Malaysian Ringgit",
	"NOK": "Norwegian Krone",
	"NZD": "New Zealand Dollar",
	"PHP": "Philippine Peso",
	"PLN": "Polish Złoty",
	"RON": "Romanian Leu",
	"SEK": "Swedish Krona",
	"SGD": "Singapore Dollar",
	"THB": "Thai Baht",
	"TRY": "Turkish Lira",
	"USD": "United States Dollar",
    "ZAR": "South African Rand"
}

from_currencies = list(CURRENCIES.keys())

to_currencies = ["EUR"]

# class ExchangeRateManager:
#     def __init__(self, api_root, currencies):
#         self.api_root = api_root
#         self.currencies = currencies
    
#     def get_exchange_rates(date, from_currency, to_currency):
#         url = api_root + date + "?from=" + from_currency + "&to=" + to_currency
#         r = requests.get(api_root)
#         print(url, r.status_code)

def get_exchange_rates(date, from_currency, to_currency):
    url = API_ROOT + date + "?from=" + from_currency + "&to=" + to_currency
    r = requests.get(url)
    print(f'Requested url: {url} with status code {r.status_code}')
    return r

In [34]:
dates = ['2024-02-12', '2024-02-13']
ds = '2024-02-12'

# for date in dates:
#     for from_currency in from_currencies:
#         for to_currency in to_currencies:
#             get_exchange_rates(date, from_currency, to_currency)

resp = get_exchange_rates(ds, from_currencies[0], to_currencies[0])
resp.json()
        

Requested url: https://api.frankfurter.app/2024-02-12?from=AUD&to=EUR with status code 200


{'amount': 1.0, 'base': 'AUD', 'date': '2024-02-12', 'rates': {'EUR': 0.60588}}